In [1]:
from bs4 import BeautifulSoup
import requests
import re

In [2]:
import string

base_url: string = "https://www.lottomaxnumbers.com"
lotto_max_past_numbers_page = requests.get(f"{base_url}/past-numbers").text

html = BeautifulSoup(lotto_max_past_numbers_page, "html.parser")


In [3]:
years_sub_url = list(map(lambda a: f"{base_url + a['href']}" , html.find(class_="yearList").find_all("a")))
#years_sub_url = list(map(lambda a_tag: a_tag.text, html.find(class_="yearList").find_all("a")))
print(years_sub_url)


['https://www.lottomaxnumbers.com/numbers/2022', 'https://www.lottomaxnumbers.com/numbers/2021', 'https://www.lottomaxnumbers.com/numbers/2020', 'https://www.lottomaxnumbers.com/numbers/2019', 'https://www.lottomaxnumbers.com/numbers/2018', 'https://www.lottomaxnumbers.com/numbers/2017', 'https://www.lottomaxnumbers.com/numbers/2016', 'https://www.lottomaxnumbers.com/numbers/2015', 'https://www.lottomaxnumbers.com/numbers/2014', 'https://www.lottomaxnumbers.com/numbers/2013', 'https://www.lottomaxnumbers.com/numbers/2012', 'https://www.lottomaxnumbers.com/numbers/2011', 'https://www.lottomaxnumbers.com/numbers/2010', 'https://www.lottomaxnumbers.com/numbers/2009']


In [4]:
select_year_doc = BeautifulSoup(requests.get(years_sub_url[0]).text, "html.parser")
trs = select_year_doc.table.find_all("tr")

for i, tr in enumerate(trs):
    if not tr.find("ul", class_="balls"):
        trs.pop(i)
print(trs)

[<tr>
<td class="noBefore colour">
<a href="/numbers/lotto-max-result-09-23-2022" title="View the Lotto Max Numbers for September 23 2022">September 23 2022</a>
<br/><div class="extra">With Max Millions!</div>
</td>
<td class="noBefore">
<ul class="balls">
<li class="ball ball">15</li>
<li class="ball ball">17</li>
<li class="ball ball">21</li>
<li class="ball ball">24</li>
<li class="ball ball">28</li>
<li class="ball ball">32</li>
<li class="ball ball">43</li>
<li class="ball bonus-ball">27</li>
</ul>
</td>
<td class="jackpot" data-title="jackpot">$70,000,000</td>
</tr>, <tr class="alt">
<td class="noBefore colour">
<a href="/numbers/lotto-max-result-09-20-2022" title="View the Lotto Max Numbers for September 20 2022">September 20 2022</a>
<br/><div class="extra">With Max Millions!</div>
</td>
<td class="noBefore">
<ul class="balls">
<li class="ball ball">9</li>
<li class="ball ball">10</li>
<li class="ball ball">11</li>
<li class="ball ball">20</li>
<li class="ball ball">24</li>
<li

In [5]:
from datetime import datetime, date

select_date: date = datetime.strptime("09-16-2022", "%m-%d-%Y").date()
dates_by_year = list(map(lambda tr: datetime.strptime(tr.find("a").text, "%B %d %Y").date() , trs))

print(select_date in dates_by_year)

True


In [6]:
from datetime import datetime

def formatDate(date_value):
    return str( str(datetime.strptime(date_value, "%B %d %Y").date()))

def getNumbers(li_tags):
    return list(map(lambda li: int(li.text), li_tags))


result = list(map(
    lambda tr: {"date" : formatDate(tr.a.text), "prize": float(tr.find(class_="jackpot").text[1:].replace(",", "")), "numbers": getNumbers(tr.find_all(class_="ball")), "bonus": int(tr.find(class_="bonus-ball").text)} , trs))
print(result)

[{'date': '2022-09-23', 'prize': 70000000.0, 'numbers': [15, 17, 21, 24, 28, 32, 43, 27], 'bonus': 27}, {'date': '2022-09-20', 'prize': 70000000.0, 'numbers': [9, 10, 11, 20, 24, 31, 48, 36], 'bonus': 36}, {'date': '2022-09-16', 'prize': 70000000.0, 'numbers': [2, 13, 24, 34, 39, 40, 46, 23], 'bonus': 23}, {'date': '2022-09-13', 'prize': 65000000.0, 'numbers': [9, 10, 14, 32, 35, 36, 46, 47], 'bonus': 47}, {'date': '2022-09-09', 'prize': 60000000.0, 'numbers': [1, 2, 15, 18, 29, 32, 36, 9], 'bonus': 9}, {'date': '2022-09-06', 'prize': 55000000.0, 'numbers': [16, 22, 28, 30, 41, 46, 47, 10], 'bonus': 10}, {'date': '2022-09-02', 'prize': 50000000.0, 'numbers': [8, 17, 21, 31, 34, 37, 39, 7], 'bonus': 7}, {'date': '2022-08-30', 'prize': 40000000.0, 'numbers': [5, 8, 12, 23, 39, 41, 48, 50], 'bonus': 50}, {'date': '2022-08-26', 'prize': 30866952.8, 'numbers': [3, 11, 22, 26, 29, 46, 48, 28], 'bonus': 28}, {'date': '2022-08-23', 'prize': 25000000.0, 'numbers': [9, 13, 14, 19, 37, 43, 50, 4]

In [11]:
from datetime import date, datetime

sample_date: date = datetime.strptime("2022-09-23", "%Y-%m-%d").date().strftime("%m-%d-%Y")
stats_page = requests.get(f"{base_url}/numbers/lotto-max-result-{sample_date}").text
html_content = BeautifulSoup(stats_page, "html.parser")
summary = html_content.find(class_="prizeStatsBox").find_all(class_="box")
summary_content = [div.find(class_="contentBox") for div in summary]

print(summary_content[0].find(class_="statSmall").text.split(": ")[1])


$49,389,100


In [12]:
number_matched_table = html_content.find(class_="atlanticBox").tbody.find_all("tr")

number_matched_table.pop()
print(number_matched_table)

#tds_values = list(map(lambda tr: tr.find_all("td"), number_matched_table))
#print(tds_values)


[<tr class="alt">
<td class="noBefore" style="text-align: left; ">
<img alt="Balls" height="23" src="/images/balls/match-7.svg" width="200"/>
<strong>Match 7</strong>
</td>
<td data-title="Prize Per Winner" style="text-align: right; ">
								$70,000,000.00
								
							</td>
<td data-title="Winners" style="text-align: right; ">
								0
								
									<br/>
</td>
<td data-title="Prize Fund" style="text-align: right; ">
							$0.00
							
							</td>
</tr>, <tr>
<td class="noBefore" style="text-align: left; ">
<img alt="Balls" height="23" src="/images/balls/match-6-plus-bonus.svg" width="200"/>
<strong>Match 6 plus Bonus</strong>
</td>
<td data-title="Prize Per Winner" style="text-align: right; ">
								$498,174.60
								
							</td>
<td data-title="Winners" style="text-align: right; ">
								0
								
									<br/>
<div class="regionWinners">
<div class="region">Quebec: <span>1</span></div>
</div>
</td>
<td data-title="Prize Fund" style="text-align: right; ">
					

In [9]:
def _total_by_region(region: list[str]):
   return {"region": region[0], "total": int(region[1])}

for tds in tds_values:
   print(tds[0].strong.text)
   print(tds[1].text.strip())
   if len(tds[2].find_all(class_="regionWinners")) != 0:
      region_winners = list(map(lambda div: div.find(class_="region").text , tds[2].find_all(class_="regionWinners")))
      print(list(map(lambda region: _total_by_region(region.split(": ")), region_winners)))
      total = sum(map(lambda region: int(region.split(": ")[1]), region_winners))
      print(region_winners)
      print(f"total: {total}")
   elif tds[2].find("span") is not None:
      print(tds[2].text.strip().replace(" ", "").split("-")[1])
   else:
      print(tds[2].text.strip())
   

NameError: name 'tds_values' is not defined